In [1]:
%autosave 0
import requests
import io

from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))



## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,location,date,total_cases,new_cases,total_deaths,new_deaths,total_cases_per_million,new_cases_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cvd_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_100k
59,ABW,Aruba,2020-05-21,101,0,3,0,945.994,0.000,28.099,...,13.085,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN
192,AFG,Afghanistan,2020-05-21,8145,492,187,9,209.231,12.639,4.804,...,2.581,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5
253,AGO,Angola,2020-05-21,52,2,3,1,1.582,0.061,0.091,...,2.405,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-20,France,143936.0,0.0,28056.0,0.0,0.0,0.0
0,2020-05-20,China,84137.0,0.0,4657.0,0.0,0.0,0.0
0,2020-05-20,Italy,227241.0,0.0,32281.0,0.0,0.0,0.0
0,2020-05-20,Spain,232289.0,0.0,27831.0,0.0,0.0,0.0
0,2020-05-20,United States,1552031.0,0.0,92830.0,0.0,0.0,0.0
0,2020-05-20,World,4957706.0,0.0,326147.0,0.0,0.0,0.0
0,2020-05-20,United Kingdom,251816.0,0.0,35552.0,0.0,0.0,0.0
0,2020-05-20,Germany,176577.0,0.0,8164.0,0.0,0.0,0.0
0,2020-05-20,Iran,126889.0,0.0,7193.0,0.0,0.0,0.0
0,2020-05-20,Turkey,152677.0,0.0,4225.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")

In [6]:
## concatenate all df in 1
frames = [follow_df1,follow_df2,follow_df3,follow_df4,follow_df5,follow_df6,follow_df7,follow_df8,follow_df9,follow_df10]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-05-21,France,143936.0,143845.0,28056.0,28132.0,91.0,-76.0
0,2020-05-21,China,84137.0,84067.0,4657.0,4638.0,70.0,19.0
0,2020-05-21,Italy,227241.0,227364.0,32281.0,32330.0,-123.0,-49.0
0,2020-05-20,Spain,232289.0,232555.0,27831.0,27888.0,-266.0,-57.0
0,2020-05-21,United States,1552031.0,1551853.0,92830.0,93439.0,178.0,-609.0
0,2020-05-21,World,4957706.0,4960975.0,326147.0,327904.0,-3269.0,-1757.0
0,2020-05-21,United Kingdom,251816.0,248293.0,35552.0,35704.0,3523.0,-152.0
0,2020-05-21,Germany,176577.0,176752.0,8164.0,8147.0,-175.0,17.0
0,2020-05-21,Iran,126889.0,126949.0,7193.0,7183.0,-60.0,10.0
0,2020-05-21,Turkey,152677.0,152587.0,4225.0,4222.0,90.0,3.0


In [7]:
## dump dataset for use it in other nb
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")